In [1]:
project_id = 'elife-data-pipeline'
source_dataset = 'de_dev'
output_dataset = 'de_dev'
output_table_prefix = 'data_science_'
manuscript_min_tf = 10
manuscript_max_tf = 0.9
state_path = 's3://ci-elife-data-pipeline/airflow-config/keyword-extraction/state-dev'

In [2]:
import os
from functools import partial
from itertools import groupby
from typing import List, Tuple, TypeVar

import joblib
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import data_science_pipeline.configure_warnings

from data_science_pipeline.sql import get_sql
from data_science_pipeline.utils.bq import to_gbq
from data_science_pipeline.utils.io import load_object_from
from data_science_pipeline.utils.misc import identity_fn
from data_science_pipeline.utils.jupyter import (
    read_big_query as _read_big_query
)
from data_science_pipeline.peerscout.models import (
    WeightedKeywordModel
)

In [3]:
model_path = os.path.join(state_path, 'senior_editor_model.joblib')
recommendation_output_table_name = '{output_dataset}.{prefix}{suffix}'.format(
    output_dataset=output_dataset,
    prefix=output_table_prefix,
    suffix='editor_recommendation'
)

In [4]:
print('loading model from:', model_path)
model_dict = load_object_from(model_path)
model_dict.keys()

loading model from: s3://ci-elife-data-pipeline/airflow-config/keyword-extraction/state-dev/senior_editor_model.joblib


dict_keys(['editor_tf_idf_vectorizer', 'editor_tf_idf', 'editor_names', 'editor_person_ids'])

In [5]:
editor_tf_idf_vectorizer = model_dict['editor_tf_idf_vectorizer']
editor_tf_idf_vectorizer

TfidfVectorizer(lowercase=False, smooth_idf=False, token_pattern=None,
                tokenizer=<function identity_fn at 0x7fee77561dd0>)

In [6]:
editor_tf_idf = model_dict['editor_tf_idf']
editor_tf_idf

<65x13658 sparse matrix of type '<class 'numpy.float64'>'
	with 218399 stored elements in Compressed Sparse Row format>

In [7]:
editor_names = model_dict['editor_names']
editor_names

0           Aleksandra Walczak
1                  Andrew King
2               Anna Akhmanova
3     Barbara Shinn-Cunningham
4                Carla Rothlin
                ...           
60                 Tamar Makin
61             Timothy Behrens
62              Utpal Banerjee
63              Vivek Malhotra
64               Wendy Garrett
Name: name, Length: 65, dtype: object

In [8]:
editor_person_ids = model_dict['editor_person_ids']
editor_person_ids

0     50904
1     14601
2      8518
3     19576
4     44396
      ...  
60    18331
61     1044
62     1042
63     1133
64    28627
Name: person_id, Length: 65, dtype: object

In [9]:
editor_person_id_by_name_map = dict(zip(editor_names, editor_person_ids))
editor_person_id_by_name_map

{'Aleksandra Walczak': '50904',
 'Andrew King': '14601',
 'Anna Akhmanova': '8518',
 'Barbara Shinn-Cunningham': '19576',
 'Carla Rothlin': '44396',
 'Catherine Dulac': '1014',
 'Chris Baker': '28129',
 'Christian Büchel': '16197',
 'Christian Hardtke': '1102',
 'Christian Rutz': '15332',
 'Clifford Rosen': '48282',
 'Cynthia Wolberger': '5005',
 'David Ron': '1174',
 'Detlef Weigel': '1030',
 'Didier Stainier': '7189',
 'Diethard Tautz': '1191',
 'Dominique Soldati-Favre': '41647',
 'Eduardo Franco': '1086',
 'Edward Morrisey': '55645',
 'Floris de Lange': '28130',
 'Gary Westbrook': '1202',
 'George Perry': '42011',
 'Gisela Storz': '1188',
 'Huda Zoghbi': '1029',
 'James Manley': '1020',
 'Jessica Tyler': '1421',
 'John Huguenard': '13947',
 'John Kuriyan': '1018',
 'Jonathan Cooper': '1062',
 'Jos van der Meer': '41027',
 'Joshua Gold': '17965',
 'José Faraldo-Gómez': '13987',
 'K VijayRaghavan': '1027',
 'Karla Kirkegaard': '3645',
 'Kate Wassum': '33127',
 'Kathryn Cheah': '91149

In [10]:
weighted_keyword_valid_model = WeightedKeywordModel.from_tf_matrix(
    editor_tf_idf.todense(),
    vectorizer=editor_tf_idf_vectorizer,
    choices=editor_names
)
weighted_keyword_valid_model

In [11]:
read_big_query = partial(_read_big_query, project_id=project_id)

In [12]:
default_query_props = dict(project=project_id, dataset=source_dataset)

In [13]:
manuscript_version_for_recommendation_df = read_big_query(
    get_sql('manuscript-version-initial-submissions-for-senior-editor-recommendation.sql').format(
        **default_query_props
    )
)
manuscript_version_for_recommendation_df.head()

> ```sql
> WITH t_manuscript_version_abstract_keywords AS (
>   SELECT
>     manuscript_abstract_keywords.manuscript_id AS manuscript_id,
>     manuscript_abstract_keywords.version_id,
>     manuscript_abstract_keywords.extracted_keywords,
>     ROW_NUMBER() OVER (
>       PARTITION BY version_id
>       ORDER BY data_hub_imported_timestamp DESC
>     ) AS version_id_row_number
>   FROM `elife-data-pipeline.de_dev.manuscript_abstract_keywords` AS manuscript_abstract_keywords
>   WHERE ARRAY_LENGTH(extracted_keywords) > 0
> ),
> 
> t_last_manuscript_version_abstract_keywords AS (
>   SELECT
>     * EXCEPT(version_id_row_number)
>   FROM t_manuscript_version_abstract_keywords
>   WHERE version_id_row_number = 1
>   ORDER BY version_id
> )
> 
> SELECT version.version_id, manuscript_version_abstract_keywords.extracted_keywords
> FROM `elife-data-pipeline.de_dev.mv_manuscript_version` AS version
> JOIN t_last_manuscript_version_abstract_keywords AS manuscript_version_abstract_keywords
>   ON manuscript_version_abstract_keywords.version_id = version.version_id
> WHERE version.overall_stage = 'Initial Submission'
>   AND ARRAY_LENGTH(version.senior_editors) = 0
>   AND TIMESTAMP_DIFF(CURRENT_TIMESTAMP, version.created_timestamp, DAY) < 365
> ```

Downloading: 100%|██████████| 84/84 [00:00<00:00, 162.31rows/s]


,version_id,extracted_keywords
0,53743/2019-11-18T20:15:11Z,"[11 different genus, 18-month, 18-month time p..."
1,55118/2020-01-13T13:01:28Z,"[15 dpf, 30 dpf, 7 dpf, abnormal, abnormal bra..."
2,51145/2019-08-16T03:29:51Z,"[, - mouse, - mice, - mice study, - mouse, 129..."
3,52166/2019-09-24T04:34:38Z,"[activity, anesthetize, anesthetized mouse, aw..."
4,52332/2019-09-30T11:45:20Z,"[584e, 584e mutation, 592a, 592a mutation, aci..."


In [14]:
keyword_similarity = cosine_similarity(
    editor_tf_idf_vectorizer.transform(
        manuscript_version_for_recommendation_df
        ['extracted_keywords']
    ),
    editor_tf_idf
)
print(keyword_similarity.max())
keyword_similarity

0.3135499769748734


array([[0.11699372, 0.05622423, 0.06988928, ..., 0.06029717, 0.05739857,
        0.17489072],
       [0.12647419, 0.11752432, 0.14188351, ..., 0.1330748 , 0.1173395 ,
        0.13240319],
       [0.1351696 , 0.11958323, 0.16990938, ..., 0.16870392, 0.15652775,
        0.17596693],
       ...,
       [0.10467359, 0.07627065, 0.1781165 , ..., 0.11683427, 0.22295422,
        0.11943398],
       [0.        , 0.00497416, 0.00252182, ..., 0.        , 0.00126713,
        0.        ],
       [0.07887325, 0.06238781, 0.08805799, ..., 0.06408936, 0.06814098,
        0.05382512]])

In [15]:
weighted_keyword_valid_model.predict_ranking(
    manuscript_version_for_recommendation_df['extracted_keywords'][:1],
).proba_matrix

matrix([[0.11699372, 0.05622423, 0.06988928, 0.05416927, 0.05546435,
         0.07196858, 0.03351713, 0.07477926, 0.07033402, 0.06574221,
         0.04553737, 0.08056315, 0.0531592 , 0.17191053, 0.06837033,
         0.16259108, 0.0869201 , 0.14647104, 0.04014783, 0.06205187,
         0.05552019, 0.05693035, 0.22370716, 0.06782173, 0.07919553,
         0.08843535, 0.04055877, 0.07710914, 0.05288817, 0.09712821,
         0.05761795, 0.05022449, 0.06833026, 0.10955343, 0.05221075,
         0.06142685, 0.04507964, 0.08535173, 0.05974032, 0.06179675,
         0.04443816, 0.0634508 , 0.10488546, 0.05949453, 0.08213281,
         0.03103112, 0.10883163, 0.25197971, 0.05394812, 0.16399081,
         0.07025179, 0.050372  , 0.07404089, 0.05702474, 0.07306   ,
         0.05452298, 0.06112094, 0.07812052, 0.04846357, 0.0740366 ,
         0.03733993, 0.06673816, 0.06029717, 0.05739857, 0.17489072]])

In [16]:
manuscript_matching_keywords_list = weighted_keyword_valid_model.predict_ranking(
    manuscript_version_for_recommendation_df['extracted_keywords']
).matching_keywords_list
pd.Series(manuscript_matching_keywords_list[:5])

0    [[(0.010062973270211556, antibiotic resistance...
1    [[(0.015022648333509846, model), (0.0075113241...
2    [[(0.019365814604007915, cell), (0.00896857043...
3    [[(0.018111620282306052, cell), (0.00932566860...
4    [[(0.007522309939932073, population), (0.00705...
dtype: object

In [17]:
T = TypeVar('T')


def get_recommended_editors_with_probability(
        proba_matrix: List[List[float]],
        manuscript_matching_keywords_list: List[List[Tuple[float, str]]],
        indices: List[T],
        threshold: float = 0.5) -> List[List[Tuple[float, T]]]:
    return [
        sorted([
            (
                p,
                key,
                sum(
                    s for s, _ in editor_matching_keywords
                ),
                editor_matching_keywords
            )
            for p, key, editor_matching_keywords in zip(
                row,
                indices,
                editors_matching_keywords
            ) if p >= threshold
        ], reverse=True)
        for row, editors_matching_keywords in zip(proba_matrix, manuscript_matching_keywords_list)
    ]


prediction_results_with_similarity = pd.Series(
    get_recommended_editors_with_probability(
        keyword_similarity,
        manuscript_matching_keywords_list,
        editor_names,
        threshold=0.001
    ),
    index=manuscript_version_for_recommendation_df.index
)
# print(prediction_results_with_similarity[0])
prediction_results_with_similarity[:5]

0    [(0.25197970778892803, Neil Ferguson, 0.251979...
1    [(0.22734899618590113, Huda Zoghbi, 0.22734899...
2    [(0.23890608606262856, Maureen Murphy, 0.23890...
3    [(0.2306392522654804, Catherine Dulac, 0.23063...
4    [(0.2481508716730846, Dominique Soldati-Favre,...
dtype: object

In [18]:
prediction_results_df = pd.concat([
    manuscript_version_for_recommendation_df['version_id'],
    prediction_results_with_similarity.to_frame('prediction'),
], axis=1)
print(len(prediction_results_df))
prediction_results_df.head()

84


,version_id,prediction
0,53743/2019-11-18T20:15:11Z,"[(0.25197970778892803, Neil Ferguson, 0.251979..."
1,55118/2020-01-13T13:01:28Z,"[(0.22734899618590113, Huda Zoghbi, 0.22734899..."
2,51145/2019-08-16T03:29:51Z,"[(0.23890608606262856, Maureen Murphy, 0.23890..."
3,52166/2019-09-24T04:34:38Z,"[(0.2306392522654804, Catherine Dulac, 0.23063..."
4,52332/2019-09-30T11:45:20Z,"[(0.2481508716730846, Dominique Soldati-Favre,..."


In [19]:
prediction_results_df['prediction'][0]

[(0.25197970778892803,
  'Neil Ferguson',
  0.2519797077889279,
  [(0.018778421250578214, 'infection'),
   (0.012516261031234347, 'transmission'),
   (0.010644762136256692, 'public'),
   (0.009314166869224604, 'antibiotic'),
   (0.008483650877752645, 'study'),
   (0.007615332991989559, 'health'),
   (0.007270938895071717, 'antibiotic resistance'),
   (0.007266631389817283, 'healthcare'),
   (0.006885237741056121, 'pathogen'),
   (0.0062635517228454285, 'multidrug-resistant'),
   (0.006240385448693939, 'isolate'),
   (0.005465307782585565, 'genome'),
   (0.005248454452949279, 'spread'),
   (0.005058904506861168, 'bacterial'),
   (0.005012988001688116, 'hospital'),
   (0.004687376771632587, 'genus'),
   (0.004536225245375354, 'genetic'),
   (0.004512789921178998, 'resistance'),
   (0.003997297722204057, 'persistence'),
   (0.003930028464985957, 'health threat'),
   (0.003756699365241932, 'virulence'),
   (0.0037217389710049974, 'method'),
   (0.003686710270934735, 'bacteria'),
   (0.0036

In [20]:
prediction_results_flat_df = pd.DataFrame([
    {
        'version_id': row.version_id,
        'score': predicted_editor[0],
        'name': predicted_editor[1],
        'person_id': editor_person_id_by_name_map[predicted_editor[1]],
        'matching_keyword_score': predicted_editor[2],
        'matching_keywords': [{
            'score': keyword_score,
            'keyword': keyword
        } for keyword_score, keyword in predicted_editor[3]],
    }
    for row in prediction_results_df.itertuples()
    for predicted_editor in row.prediction
])
print(len(prediction_results_flat_df))
prediction_results_flat_df.head()

5386


,version_id,score,name,person_id,matching_keyword_score,matching_keywords
0,53743/2019-11-18T20:15:11Z,0.251980,Neil Ferguson,12065,0.251980,"[{'score': 0.018778421250578214, 'keyword': 'i..."
1,53743/2019-11-18T20:15:11Z,0.223707,Gisela Storz,1188,0.223707,"[{'score': 0.02024155720099594, 'keyword': 'ba..."
2,53743/2019-11-18T20:15:11Z,0.174891,Wendy Garrett,28627,0.174891,"[{'score': 0.01576882434140405, 'keyword': 'in..."
3,53743/2019-11-18T20:15:11Z,0.171911,Detlef Weigel,1030,0.171911,"[{'score': 0.01445236873766115, 'keyword': 'ge..."
4,53743/2019-11-18T20:15:11Z,0.163991,Patricia Wittkopp,42157,0.163991,"[{'score': 0.016305205828397774, 'keyword': 'g..."


In [21]:
prediction_results_flat_df['version_id'].nunique()

84

In [22]:
prediction_results_flat_df.max()

version_id                58803/2020-05-11T11:55:13Z
score                                        0.31355
name                                   Wendy Garrett
person_id                                      97155
matching_keyword_score                       0.31355
dtype: object

In [23]:
print('writing to:', recommendation_output_table_name)
to_gbq(
    prediction_results_flat_df,
    recommendation_output_table_name,
    project_id=project_id,
    if_exists='replace'
)
print('done')

writing to: de_dev.data_science_editor_recommendation
done
